In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

pd.set_option("display.max_rows", 200)
pd.set_option("display.max_columns", 50)


In [7]:
def build_backtest_df(price_df: pd.DataFrame, signal: pd.Series) -> pd.DataFrame:
    df = price_df.copy().sort_index()

    # daily returns
    df["ret"] = df["Close"].pct_change().fillna(0.0)

    # align signal to df index
    sig = signal.reindex(df.index).fillna(0).astype(int)
    df["signal"] = sig

    # IMPORTANT: trade at next day close => shift signal by 1 day for position
    df["position"] = df["signal"].shift(1).fillna(0).astype(int)

    # strategy returns
    df["strategy_ret"] = df["position"] * df["ret"]

    # equity curves start at 1.0
    df["equity_buy_hold"] = (1.0 + df["ret"]).cumprod()
    df["equity_strategy"] = (1.0 + df["strategy_ret"]).cumprod()

    return df


def compute_trade_count(position: pd.Series) -> int:
    pos = position.fillna(0).astype(int)
    # entries are 0 -> 1
    entries = (pos.diff() == 1).sum()
    return int(entries)


def compute_metrics(bt: pd.DataFrame) -> dict:
    total_return = bt["equity_strategy"].iloc[-1] - 1.0
    buy_hold_return = bt["equity_buy_hold"].iloc[-1] - 1.0
    n_trades = compute_trade_count(bt["position"])

    # optional “nice to have” metrics
    daily = bt["strategy_ret"]
    vol = float(daily.std() * np.sqrt(252)) if daily.std() != 0 else np.nan
    sharpe = float((daily.mean() / daily.std()) * np.sqrt(252)) if daily.std() != 0 else np.nan
    max_dd = float(((bt["equity_strategy"] / bt["equity_strategy"].cummax()) - 1.0).min())

    return {
        "total_return_strategy": float(total_return),
        "total_return_buy_hold": float(buy_hold_return),
        "number_of_trades": int(n_trades),
        "annualized_vol": vol,
        "sharpe_approx": sharpe,
        "max_drawdown": max_dd,
    }


def plot_equity_curve(bt: pd.DataFrame, title="Equity Curve"):
    plt.figure(figsize=(12, 4))
    plt.plot(bt.index, bt["equity_strategy"], label="Strategy")
    plt.plot(bt.index, bt["equity_buy_hold"], label="Buy & Hold", alpha=0.7)
    plt.title(title)
    plt.xlabel("Date")
    plt.ylabel("Equity (start=1.0)")
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()


def plot_price_with_signals(bt: pd.DataFrame, title="Price with Buy/Sell Markers"):
    pos = bt["position"].fillna(0).astype(int)
    entry = (pos.diff() == 1)
    exit_ = (pos.diff() == -1)

    plt.figure(figsize=(12, 4))
    plt.plot(bt.index, bt["Close"], label="Close")
    plt.scatter(bt.index[entry], bt.loc[entry, "Close"], marker="^", label="Buy (enter long)")
    plt.scatter(bt.index[exit_], bt.loc[exit_, "Close"], marker="v", label="Sell (exit to cash)")
    plt.title(title)
    plt.xlabel("Date")
    plt.ylabel("Price")
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()
